In [1]:
#load libraries

import pandas as pd
import numpy as np
import glob
from pathlib import Path

In [2]:
## Cleaning the data of all mutual fund filling the non float cell by NAN
## droping the row if the mutual fund scheme is closed
## Adding mean, variance, std and sharp ratio


## path of all mutual fund's csv folder
path = r"C:\Users\Admin\Documents\Projects\mf\mf"


for file in glob.glob(path+"\\*.csv"):
    name = Path(file).name                                                          ## Name of Mutual Fund
    df = pd.read_csv(file, index_col=0)
    
    for rowIndex, row in df.iterrows():                                             ## iterate over rows
        for columnIndex, value in row.items():
            try:
                df.loc[rowIndex,columnIndex]=float(df.loc[rowIndex,columnIndex])    ## str to float
            except:
                df.loc[rowIndex,columnIndex]=np.nan                                 ## replace non float value by Nan

    df = df.astype(float)                                                           ## df type float

    df.drop(df[df.count(axis='columns') <= 6].index, inplace = True)                ## dropping the mutualfund scheme plans which has less than 6 month data
    
    df.drop(df[df.iloc[:, -1].isnull()].index, inplace = True)                      ## dropping the mutualfund scheme plans which has closed
    
    df = round(df.interpolate(method='linear',axis=1), 3)                           ## Linearly interpolating the data for filling the empty cell 

    df['mean'] = round(df.mean(axis=1), 3)                                          ## adding mean column
    df['var'] = round(df.iloc[:,:-1].var(axis=1), 3)                                ## adding var column
    df['std'] = round(df.iloc[:,:-2].std(axis=1), 3)                                ## adding std column
    
    
    df['sharpe ratio'] = round(df['mean']/df['std']*pow(12, 0.5), 3)                             ## adding sharpe ratio by neglecting the risk free rate

    df.to_csv(r'C:\Users\Admin\Documents\Projects\mf\mf_name\{}.csv'.format(name))     ## new csv for every mutual fund

In [3]:
## merged all csv files into 1 master csv file


path = r"C:\Users\Admin\Documents\Projects\mf\mf_name"            ## path for csv folder 
output = pd.DataFrame()

for file in glob.glob(path+"\\*.csv"):
    cn = pd.read_csv(file)
    output = pd.concat([output,cn])

output.to_csv(r"C:\Users\Admin\Documents\Projects\mf\master.csv", index=False) ## new merged csv

In [4]:
master_df = pd.read_csv("master.csv", index_col=0)

In [5]:
for idx in master_df.index:
    if "Growth" not in idx or "Direct" not in idx:                           ## get only one plan for every scheme
        master_df.drop([idx], axis=0, inplace=True)

In [6]:
## different equity catogeries for mutual fund's and their keywords

categories = {'Large Cap': ['Large Cap', 'Bluechip', 'Frontline'],  
              'Mid Cap':'Mid', 
              'Small Cap':'Small', 
              'Flexi Cap':'Flexi', 
              'Value Fund':'Value', 
              'Focused Fund':'Focused', 
              'Dividend Yield Fund': 'Dividend', 
              'Retirement Fund' : 'Retirement', 
              'ELSS Fund': ['ELSS', 'Tax'],
              'Index Fund' : 'Index'}

In [7]:
for catogery, sign in categories.items():                                          ## getting key and values in dict
    
    new_df = pd.DataFrame()                                                        ## dataframe for every scheme

    for idx in master_df.index:                                                    ## every raw in master dataframe
        if type(sign) == list:
            for j in range(len(sign)):
                if sign[j] in idx:
                    helper_df = pd.DataFrame(master_df.loc[[idx]])           
                    new_df = pd.concat([new_df, helper_df])                        ## making new dataframe for every scheme
        else:
            if sign in idx:
                helper_df = pd.DataFrame(master_df.loc[[idx]])
                new_df = pd.concat([new_df, helper_df])
                
    new_df.sort_values(by=['sharpe ratio', 'mean'], inplace=True, ascending=False)  ## sorting the dataframe in descending order
                
    new_df.to_csv(r"C:\Users\Admin\Documents\Projects\mf\mf_scheme\{}.csv".format(catogery)) ## making csv